In [3]:
import os
# Spécifiez le chemin du dossier racine
dossier_racine = r'C:\Users\lucie\Documents\Programmation\AAM-beatinfo'

# Liste pour stocker tous les fichiers trouvés
tous_les_fichiers = []

# Parcourir récursivement l'arborescence
for dossier, sous_dossiers, fichiers in os.walk(dossier_racine):
    for fichier in fichiers:
        chemin_fichier = os.path.join(dossier, fichier)
        tous_les_fichiers.append(chemin_fichier)



In [4]:
lignes_list = []

for fichier in tous_les_fichiers:
    with open(fichier, 'r') as fichier_ouvert:
        lignes = fichier_ouvert.readlines()[8:]  # Ignorer les 5 premières lignes
        # Faites quelque chose avec le reste des lignes du fichier
        # Par exemple, afficher les 5 lignes suivantes de chaque fichier
        lignes_list.append(lignes)
        

In [5]:
start = []
chords = []

for el in lignes_list:
    for var in el:
        val = var.split(',')
        start.append(val[0])
        chordName = val[-1]
        chordName = chordName[:-2].replace("'",'')
        chords.append(chordName)

In [6]:
import pandas as pd

df = pd.DataFrame()
df['StartTime'] = start
df['Chordsname'] = chords

df

,StartTime,Chordsname
0,0.65217394,Fmaj
1,1.3043479,Fmaj
2,1.9565217,Fmaj
3,2.6086957,A#maj
4,3.2608697,A#maj
...,...,...
293307,159.67743,D#min
293308,160.40913,F#maj
293309,161.14084,F#maj
293310,161.87254,F#maj


In [7]:
import numpy as np

np.unique(df['Chordsname']), len(np.unique(df['Chordsname']))

(array(['A#maj', 'A#min', 'Amaj', 'Amin', 'Bmaj', 'Bmin', 'C#maj', 'C#min',
        'Cmaj', 'Cmin', 'D#maj', 'D#min', 'Dmaj', 'Dmin', 'Emaj', 'Emin',
        'F#maj', 'F#min', 'Fmaj', 'Fmin', 'G#maj', 'G#min', 'Gmaj', 'Gmin',
        'N.C.'], dtype=object),
 25)

In [32]:
from mingus.core import chords
from mingus.containers import NoteContainer
from mingus.midi import midi_file_out as MidiFileOut

df['notes'] = df['Chordsname'].apply(lambda x: chords.from_shorthand(x) if 'N.C.' not in x else np.nan)
df['notes'] = df['notes'].astype(str)

In [33]:
df

,StartTime,Chordsname,notes
0,0.65217394,Fmaj,"['F', 'A', 'C']"
1,1.3043479,Fmaj,"['F', 'A', 'C']"
2,1.9565217,Fmaj,"['F', 'A', 'C']"
3,2.6086957,A#maj,"['A#', 'C##', 'E#']"
4,3.2608697,A#maj,"['A#', 'C##', 'E#']"
...,...,...,...
293307,159.67743,D#min,"['D#', 'F#', 'A#']"
293308,160.40913,F#maj,"['F#', 'A#', 'C#']"
293309,161.14084,F#maj,"['F#', 'A#', 'C#']"
293310,161.87254,F#maj,"['F#', 'A#', 'C#']"


In [34]:
nb_classe =  len(np.unique(df['Chordsname']))
classes = np.unique(df['Chordsname'])

In [35]:
import re
import tensorflow as tf

max_features = 4000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['notes'].values)
X = tokenizer.texts_to_sequences(df['notes'].values)
X = tf.keras.preprocessing.sequence.pad_sequences(X) 

In [36]:
X[0]

array([ 0,  0,  0, 14,  6,  9])

In [37]:
X.shape

(293312, 6)

In [39]:
embed_dim = 300
lstm_out = 196

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(tf.keras.layers.SpatialDropout1D(0.4))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_out, dropout=0.05, recurrent_dropout=0.2)))
model.add(tf.keras.layers.Dense(nb_classe, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 6, 300)            1200000   
                                                                 
 spatial_dropout1d_1 (Spati  (None, 6, 300)            0         
 alDropout1D)                                                    
                                                                 
 bidirectional_1 (Bidirecti  (None, 392)               779296    
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 25)                9825      
                                                                 
Total params: 1989121 (7.59 MB)
Trainable params: 1989121 (7.59 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
from sklearn.model_selection import train_test_split
import numpy as np
Y = pd.get_dummies(df['Chordsname'])
Y.head()

,A#maj,A#min,Amaj,Amin,Bmaj,Bmin,C#maj,C#min,Cmaj,Cmin,...,Emin,F#maj,F#min,Fmaj,Fmin,G#maj,G#min,Gmaj,Gmin,N.C.
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [41]:
Y = Y.values

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(196519, 6) (196519, 25)
(96793, 6) (96793, 25)


In [43]:
history = model.fit(X_train, Y_train, epochs=20, batch_size=128, verbose=2,validation_split=0.3)

Epoch 1/20
1075/1075 - 129s - loss: 0.2012 - accuracy: 0.9096 - val_loss: 0.1118 - val_accuracy: 0.9302 - 129s/epoch - 120ms/step
Epoch 2/20


KeyboardInterrupt: 